In [1]:
#from utils.paper import describe_data
#from utils import project_path

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

from utils import ROOT_DIR, get_data

%load_ext autoreload
%autoreload 2

%matplotlib inline

# Introduction

# Data

During the pandemic, spatially disaggregated (city-level) COVID-19 data have been collected in China by two major providers: the Chinese news agency “The Paper” (Pengpai News agency, 2020) and Xu et al. (2020). Meanwhile, Johns Hopkins University Center for Systems Science and Engineering (JHU) has provided an online platform to allow users to download and visualize regularly updated data worldwide. This is currently the most comprehensive database on COVID-19 worldwide aggregated at national or district/province levels (JHU CSSE, 2020; Dong, Du, and Gardner, 2020).

This study uses 2,321 observations geo-located COVID-19 data from Xu et al. (2020) gathered in China and aggregated temporally between January and March 2020. The original data is composed of positive-only COVID-19 cases (622 observations), which were complemented by 1,699 background data (also-called pseudo-absence data) following the procedure described in Barbet-Massin et al. (2012). Our response represents the (log) rate of COVID-19 infections (number of positive cases per 100,000 population) reported between January and March 2020 per grid-cell. To mitigate the effects of outliers (extreme high values of COVID-19 incidence rate per 100,000 population (Jan-Mar 2020) in Wuhan), the response is logged and can be expressed as: log COVID-19 incidence rate per 100,000 population (Jan-Mar 2020).

We consider seven relevant predictive features (Figure A1). We account for the geolocation of the observations (LONG and LAT), which can be associated with the onset and the rate of the spread of COVID-19. Cities in the east and in the north of China tended to record COVID-19 cases earlier, than those in the west and in the south (Zhang, Zhang, and Wang, 2020). In China, the initial COVID-19 outbreak is mainly explained by human movement (Kraemer et al., 2020) and distance to the initial focus presumably located in the city of Wuhan (Liu, 2020). Furthermore, given the importance of human-to-human COVID-19 transmission, densely populated areas are more at risk to encounter higher incidence rates and exhibit a larger number of total confirmed cases (Maier and Brockmann, 2020; Tian et al., 2020). Therefore, the model includes travel time to: (i) the nearest city with a population of more than 50,000 (Weiss et al., 2018) (ACCESS); and (ii) to Wuhan (WACCESS) and controls for population size (POP) using WorldPop (Tatem, 2017) and urban footprint (URBAN) (Liu, 2020; Pirouz et al., 2020) (URBAN) using the Global Urban Footprint data (Esch et al., 2017).

While climatic and meteorological factors might have played a minor role in the transmission of COVID-19, humidity could facilitate the transmission of the virus and has shown a positive correlation with the number of total confirmed cases of COVID-19 in China (Liu et al., 2020). In order to account for potential variability in air transmission, we include humidity by potential evapotranspiration (PET), which measures the potential amount of evaporation.

To improve the model stability, we transformed the features using a quantile normalization technique (Bolstad et al., 2003), except LONG (-180 to 180 degrees) and LAT (-90 to 90 degrees) which are transformed by dividing them by their respective highest absolute value. A total number of 812 non-missing observations are used for model training, validation and testing (summary statistics of the data in Table A1).

---

<!--
This study uses geo-located COVID-19 data gathered in China and temporally aggregated between January to March 2020, which consists of 2,321 observations (Xu et al., 2020). The original data is composed of positive-only COVID-19 cases (622 observations), which were complemented by 1,699 background data (also-called pseudo-absence data) following the procedure described in Barbet-Massin et al. (2012). The resulting dataset refers to as an augmented dataset. Here, background data are generated in all locations (spatially randomly generated locations separated by 0.5 degree distance) that satisfy at least one of these two conditions: (1) The travel time to large cities (ACCESS) is above the 97.5% quantile of the sample or (2) the population size (POP) is lower than the 2.5% quantile sample.

We took a purely spatial approach instead of a spatiotemporal framework, and therefore ignore the dynamic nature of COVID-19 (Hsiang et al., 2020). The modeling choice is mainly based on limitations due to the sparsity of the data at fine spatial scales in China. During the observed period, most grid-cells (at about 5 x 5 km2 resolution) in China were not affected by COVID-19. The augmentation procedure used to attribute zero to areas that cannot count positive cases (impossible positive cases), led to a very large number of zeros. Consequently, accounting for time would only increase the sparsity already present in the data and hence make spatiotemporal prediction highly challenging. Following the literature, we consider a total of seven features (Figure A1). We account for the geolocation of the observations (LONG and LAT), which can be associated with the onset and the rate of the spread of COVID-19. Cities in the east and in the north of China tended to record COVID-19 cases earlier than those in the west and in the south (Zhang, Zhang, and Wang, 2020). In China, COVID-19 outbreak can be mainly explained by human movement (Kraemer et al., 2020) and distance to Wuhan (Liu, 2020). Furthermore, given the importance of human-to-human COVID-19 transmission, densely populated areas are more at risk to encounter higher incidence rates and exhibit a larger number of total confirmed cases (Maier and Brockmann, 2020; Tian et al., 2020). Therefore, we control for travel time to: (i) the nearest city with a population of more than 50,000 (Weiss et al., 2018) (ACCESS); and (ii) to Wuhan (WACCESS) following Weiss et al. (2018) and control for population size (POP) using WorldPop (Tatem, 2017) and urban footprint (URBAN) (Liu, 2020; Pirouz et al., 2020) (URBAN) using the Global Urban Footprint data (Esch et al., 2017).

While climatic and meteorological factors might have played a minor role in the transmission of COVID-19, humidity can facilitate the transmission of the virus and has reported as positively correlated with the number of total confirmed cases of COVID-19 in China (Liu et al., 2020). In order to account for potential variability in air transmission, we include humidity by potential evapotranspiration (PET), which measures the potential amount of evaporation.
-->

There are various approaches to assess COVID-19 risk at fine spatial scale. One may model COVID-19 total confirmed cases (Jan-Mar 2020) (TC) in each grid-cell. While TC provides a relevant risk metric, modeling it directly makes its interpretation difficult. Cities with a high TC might not necessarily be more at risk than those with a lower TC, since cities with larger population size are likely to exhibit larger values of TC. In order to mitigate this issue, we normalized the target variable (response) by the adjusted population size (VAPOP) estimated for each area surrounding the observed locations delimited by Voronoi polygons (Burrough et al., 2015, pp. 160) and computed the incidence rate per 100,000 (IR) (Equation A1). Further, to mitigate the effects of outliers (extreme high values of IR in Wuhan), we define the resulting target variable as logged IR (LIR, see Equation A2).

\begin{equation}
IR = \dfrac{TC}{VAPOP} \times 100,000 \qquad (A1)
\end{equation}

\begin{equation}
LIR = \ln (IR + 1) \qquad (A2)
\end{equation}

In order to improve the model stability, we transformed the features using a quantile normalization technique (Bolstad et al., 2003), except LONG (-180 to 180 degrees) and LAT (-90 to 90 degrees) which are transformed by dividing them by their respective highest absolute value. As a result, LONG and LAT values range lie between zero and one, which avoid numerical issues (Bengio, 2012). The complete dataset (812 observations without missing values) is used for model training, validation and testing (summary in Table A1).

In [2]:
print("Table A1.: Summary statistics of the data used in the model. The features include:geographic coordinates longitude (LONG) and latitude (LAT), travel time to the nearestcity of with a population exceeding 50,000 (ACCESS), potential evapotranspiration(PET), population size (POP), urbanity (URBAN) and travel time to Wuhan(WACCESS). The target variable (response) is (8) log COVID-19 incidence rate per100,000 population (Jan-Mar 2020) (LIR). LONG and LONG values are divided by theirabsolute value and the other features are transformed with a quantile normalizationtechnique (Bolstad et al., 2003).", "\n")
# Summary of data from "/data/csvs/data.csv" (http://c100-159.cloud.gwdg.de:9009/lab/tree/data/csvs/data.csv)
# describe_data()

Table A1.: Summary statistics of the data used in the model. The features include:geographic coordinates longitude (LONG) and latitude (LAT), travel time to the nearestcity of with a population exceeding 50,000 (ACCESS), potential evapotranspiration(PET), population size (POP), urbanity (URBAN) and travel time to Wuhan(WACCESS). The target variable (response) is (8) log COVID-19 incidence rate per100,000 population (Jan-Mar 2020) (LIR). LONG and LONG values are divided by theirabsolute value and the other features are transformed with a quantile normalizationtechnique (Bolstad et al., 2003). 



In [3]:
raw_data = pd.read_csv(ROOT_DIR + "/data/csvs/rawdata.csv")
raw_data.head()

,id,LONG,LAT,TC,voronoi_pop,nbHF,VAPOP,sampling,provnames,ACCESS,PET,POP,URBAN,WACCESS
0,1,109.5792,18.29187,4.0,294324.531250,1,294324.531250,1,Hainan,-0.625080,1.346743,2.067364,1.284664,2.655904
1,2,109.7180,18.36956,2.0,73959.992188,1,73959.992188,1,Hainan,NaN,NaN,NaN,NaN,NaN
2,3,109.4197,18.39084,60.0,218175.828125,1,218175.828125,1,Hainan,-0.737850,1.160834,0.871805,0.716921,2.674515
3,4,109.9475,18.56796,6.0,322905.031250,1,322905.031250,1,Hainan,-0.523366,1.269342,1.113280,0.613657,2.634486
4,5,109.0491,18.65009,4.0,465194.500000,1,465194.500000,1,Hainan,-1.265526,1.233569,0.818068,0.533877,2.691532


In [4]:
raw_data.query("ACCESS >= ACCESS.quantile(0.95)").describe()

,id,LONG,LAT,TC,voronoi_pop,nbHF,VAPOP,sampling,ACCESS,PET,POP,URBAN,WACCESS
count,116.000000,116.000000,116.000000,0.0,1.160000e+02,116.0,1.160000e+02,116.0,116.000000,116.000000,116.000000,116.000000,116.000000
mean,488.000000,86.595187,34.558549,NaN,5.672673e+05,1.0,5.672673e+05,0.0,2.084007,-0.771653,-1.525421,-0.473654,1.164811
std,75.437161,3.415915,1.624140,NaN,8.879940e+05,0.0,8.879940e+05,0.0,0.367594,0.582523,0.407935,0.095290,0.433135
min,181.000000,78.270833,29.520833,NaN,1.039871e+05,1.0,1.039871e+05,0.0,1.652304,-2.363020,-1.974678,-0.482502,0.169923
25%,481.000000,84.364583,33.270833,NaN,1.039871e+05,1.0,1.039871e+05,0.0,1.819137,-1.159394,-1.974678,-0.482502,0.790319
50%,481.000000,86.395833,34.520833,NaN,5.158962e+05,1.0,5.158962e+05,0.0,1.962916,-0.667736,-1.505685,-0.482502,1.147461
75%,559.000000,88.895833,35.770833,NaN,5.158962e+05,1.0,5.158962e+05,0.0,2.262050,-0.367431,-1.305703,-0.482502,1.457051
max,559.000000,101.395833,37.645833,NaN,8.842163e+06,1.0,8.842163e+06,0.0,3.444752,0.387251,-0.099741,0.543802,2.391888


In [5]:
raw_data.query("POP <= POP.quantile(0.05)").describe()

,id,LONG,LAT,TC,voronoi_pop,nbHF,VAPOP,sampling,ACCESS,PET,POP,URBAN,WACCESS
count,116.000000,116.000000,116.000000,0.0,1.160000e+02,116.0,1.160000e+02,116.0,116.000000,116.000000,1.160000e+02,116.000000,116.000000
mean,455.637931,91.805316,38.319325,NaN,1.010822e+06,1.0,1.010822e+06,0.0,1.212095,0.313605,-1.974678e+00,-0.448281,0.669802
std,90.088401,5.639838,2.544643,NaN,8.760660e+05,0.0,8.760660e+05,0.0,0.639987,1.397401,8.920317e-16,0.183107,0.506022
min,349.000000,80.770833,33.270833,NaN,1.039871e+05,1.0,1.039871e+05,0.0,-0.414080,-2.363020,-1.974678e+00,-0.482502,-0.267109
25%,352.000000,88.895833,36.395833,NaN,1.039871e+05,1.0,1.039871e+05,0.0,0.653575,-0.997434,-1.974678e+00,-0.482502,0.375073
50%,481.000000,91.083333,38.270833,NaN,5.158962e+05,1.0,5.158962e+05,0.0,1.308815,0.214757,-1.974678e+00,-0.482502,0.588537
75%,559.000000,95.770833,40.145833,NaN,1.885811e+06,1.0,1.885811e+06,0.0,1.766923,1.625414,-1.974678e+00,-0.482502,0.791633
max,603.000000,104.520833,45.770833,NaN,3.051850e+06,1.0,3.051850e+06,0.0,2.424813,2.747000,-1.974678e+00,0.681398,2.028019


In [6]:
# replace TC NaN values with 0 values, when ACCESS is higher than 95 Percentile of ACCESS values
raw_data_transformed = raw_data.copy()
raw_data_transformed["TC"] = np.where(raw_data_transformed["ACCESS"] >= raw_data_transformed["ACCESS"].quantile(0.95), 0, raw_data_transformed["TC"])
raw_data_transformed["TC"] = np.where(raw_data_transformed["POP"] <= raw_data_transformed["POP"].quantile(0.05), 0, raw_data_transformed["TC"])

# number of zero values
print("Number of TC = 0:", len(raw_data_transformed.query("TC == 0")))


Number of TC = 0: 193


In [7]:
# Drop NA data points
raw_data_transformed = raw_data_transformed.dropna()
raw_data_transformed.describe()

,id,LONG,LAT,TC,voronoi_pop,nbHF,VAPOP,sampling,ACCESS,PET,POP,URBAN,WACCESS
count,812.000000,812.000000,812.000000,812.000000,8.120000e+02,812.000000,8.120000e+02,812.000000,812.000000,812.000000,812.000000,812.000000,812.000000
mean,343.961823,107.498655,33.592855,50.710591,1.949372e+06,1.019704,1.928856e+06,0.762315,-0.481589,0.118251,0.672332,0.788644,-0.429321
std,172.787054,12.152613,6.278269,474.470778,1.949150e+06,0.139068,1.938481e+06,0.425927,1.409672,0.781895,1.576832,1.164337,1.106955
min,1.000000,78.270833,18.291870,0.000000,1.699957e+04,1.000000,1.699957e+04,0.000000,-3.487341,-2.363020,-1.974678,-0.482502,-3.676348
25%,199.750000,101.395833,29.521206,2.000000,5.158962e+05,1.000000,5.158962e+05,1.000000,-1.493258,-0.399133,-0.216299,-0.482502,-1.127073
50%,365.500000,109.904050,34.279290,4.000000,1.368634e+06,1.000000,1.368634e+06,1.000000,-0.734903,0.173306,0.907536,0.798634,-0.720350
75%,481.000000,116.805175,37.645833,22.000000,2.589056e+06,1.000000,2.580963e+06,1.000000,0.301102,0.575787,1.758941,1.581750,0.308398
max,614.000000,132.546400,49.717850,13243.000000,1.390106e+07,2.000000,1.390106e+07,1.000000,3.444752,2.789949,4.630500,4.335167,2.691532


In [8]:
# Add a new variable IR which is (TC/VAPOP)*100,000
raw_data_transformed["IR"] = (raw_data_transformed["TC"] / raw_data_transformed["VAPOP"]) * 100000


In [9]:
# Drop not used variables from the table and set this table to data, 
# which will be used for other calculations
data = raw_data_transformed.drop(["id", "voronoi_pop", "VAPOP", "nbHF", "TC", "sampling", "provnames"], axis=1)


In [10]:
# Standardize data
data["IR"] = np.log(data["IR"] + 1)
data["LONG"] = data["LONG"] / 180
data["LAT"] = data["LAT"] / 90

In [11]:
def train_val_test_split(X, Y):
    random_state = 201
    # check data has been read in properly
    _X_train, X_test, _Y_train, Y_test = train_test_split(
        X, Y, test_size=0.2, random_state=random_state)

    X_train, X_validation, Y_train, Y_validation = train_test_split(
        _X_train, _Y_train, test_size=0.25, random_state=random_state)  # 0.25 x 0.8 = 0.2
    return X_train, Y_train, X_validation, Y_validation, X_test, Y_test

In [12]:

# create a dataframe with all training data except the target column
X = data.drop(columns=["IR"])

# create a dataframe with only the target column
Y = data[["IR"]]

# The same as get_data() from train.csv, validation.csv and test.csv
X_train, Y_train, X_validation, Y_validation, X_test, Y_test = train_val_test_split(X, Y)


# Method

## Multilayer perceptron implementation

We consider a multilayer perceptron (MLP), which is a class of DL models (Ramchoun et al., 2016). We use this model to predict COVID-19 within grid-cells of about 5 x 5 km<sup>2</sup> resolution across mainland China. The artificial neural network (ANN) architecture of the model allows us to learn complex non-linear relationships and interactions that are expected between socioeconomic and environmental predictors and COVID-19 (Yang, Jiang, and Guo, 2019).

We evaluate the predictive performance of the model during three phases: (1) training (model building); (2) validation (tuning); and (3) testing. To do so, we split the data randomly into three parts: training (60%), validation (20%), and testing (20%). We ensure that data used in the final testing has not been used to build or train the model. We use the mean squared error loss function to assess the model predictive performance, which is commonly used for this type of target variable (log LIR). We provide further details on the choice of the score metric in Appendix A.

The MLP model can be composed of zero, one or several hidden layers. Each hidden layer is composed of hidden units. We use a non-linear activation function (rectified linear unit) for the hidden units to learn the relationship between the features and the target (Brownlee, 2018, p. 141), which is expected to be complex and non-linear in the context of contagious diseases (Bhatt et al., 2017). To decrease the training time and improve the model performance and stability, we adopt batch normalization (BN), which is a widely-used technique to normalize z values in hidden layers (Goodfellow, Bengio, and Courville, 2016, pp. 168). The z values are defined as the sum of the current neuron bias and aggregation of the current neuron weights with the activation values from the previous layer. BN has a slight regularization effect and decreases the impact of weight initialization on model performance (Ioffe and Szegedy, 2015). We use a batch size of 64, as suggested in Goodfellow, Bengio, and Courville (2016). To reduce overfitting and improve generalization error we randomly drop out nodes during training (dropout regularization, see Appendix A.2.1). To optimize the MLP models we use the Adam optimizer, which can achieve high predictive performance while reducing the computation costs (Kingma and Ba, 2015). More details on the multilayer perceptron (MLP) and the choices made to parameterize the model are provided in Appendix A.

In [13]:
# Hier must be logged maps as on p. 15

## Model implementation and parametrization

We use a cloud server with 32 processor cores and 128 gigabyte random-access memory (RAM) to run the investigated DL (MLP) model. For MLP, the choice of the number of layers and hidden layers, along with the main parameters are described in further details below.

To evaluate the model while building and tuning the model, we split the data randomly into three parts: training (60%), validation (20%), and test (20%) categories using the Python library scikit-learn (Pedregosa et al., 2011). Note that the proportion of the data uses for each category could be amended according to the study context. For very large datasets, the training set can include a larger proportion of observations, since a small proportion used for validation and test may still represent a large number of observations in absolute terms, and can therefore suffice to assess the predictive performance of the investigated models.

Recall that our target variable log COVID-19 incidence rate per 100,000 population (Jan-Mar 2020) (LIR) is a continuous scaled variable and our predictive study refers to a regression problem. Hence, a suitable function should be selected to optimize the model. Common loss functions include the mean squared error (MSE), mean squared logarithmic error (MSLE), and the mean absolute error (MAE). MSE is the average of squared differences between true target values and predicted values. As all errors are squared, it disproportionately penalizes models producing larger errors. This is particularly useful in the presence of outliers. By construction (Equation A3), MSLE does not strongly penalize large differences between predictions and observations, which could be appropriate with unscaled targets (Brownlee, 2018, p. 60).

 \begin{equation}
    MSLE = \dfrac{1}{N} \sum_{i=1}^{N}(\ln(y_{i} + 1) - \ln({\hat{y}}_{i} + 1))^{2} \qquad (A3)
\end{equation}

Alternatively, the MAE loss is often used when the distribution of the target variable is approximately Gaussian, but has outliers (Brownlee, 2018, p. 62). By measuring the sum of absolute differences between true and predicted target variable values, the metric is more robust to outliers (Equation A4) and is suitable for target variable that are not scaled. 

\begin{equation}
MAE =  \dfrac{1}{N} \sum_{i=1}^{N} |y_{i} - \hat{y}_{i}| \qquad (A4)
\end{equation}

We use the MSE loss, because the target variable (LIR) is scaled and therefore exhibit a small number of outliers. However, some outliers remain (e.g. very large LIR in Wuhan, where COVID-19 was initially identified) and need to be well estimated to provide sufficiently accurate and reliable predictive maps for policy-makers.

#### Parameterizing the multilayer perceptron (MLP) model

There are various aspects that need to be considered to parameterize the MLP model. First, the number of hidden layers needs to be chosen. There is no a priori rule to determine the number of hidden layers, but parsimony should be favored. Models with a large number of hidden layers are more likely to lead to overfitting issues and are computationally more expensive (Brownlee, 2018, p. 23). Furthermore, Woods and Bowyer (1997) demonstrates that any complex and non-linear function can be estimated with a single hidden layer. To assess the effect of the number of layers on the predictive performance of the MLP model, we specify three parsimonious models: (1) without a hidden layer; (1) with one hidden layer; and (3) with two hidden layers.

Each hidden layer consists of hidden units. Models tend to perform better with a high ratio of hidden units per hidden layer and a constant ratio across each hidden layer (Bengio, 2012; Larochelle et al., 2009), but may vary according to the dataset. In addition, it is recommended to have a larger number of hidden units than features (input units) although, increasing the number number of hidden units leads to higher computational costs (Bengio, 2012). Following these recommendations, we initialize our MLP with 128 hidden units and evaluate the predictive performance with variations of the number of hidden units.

We associate non-linear activation function to hidden units to learn the relationship between the feature and the target, which is expected to be complex and non-linear (Brownlee, 2018, p. 141). There are three commonly-used activation functions: the sigmoid function, the hyperbolic tangent function (TanH) and the rectified linear unit (ReLU). The choice of the activation function depends on the context. Sigmoid and TanH activation functions have an S-shaped curve, but exhibit a different range of output values. By construction, below or above a certain threshold, the output saturates to a fixed value. Both activation functions are however sensitive to changes for small input values close to 0. As a result, training is computationally expensive and models with these activation functions often may not perform well.

In order to overcome the saturation problem, we chose ReLU as activation function. It is a non-linear function that is sensitive to the entire range of positive input values, but also enables the model to learn complex relationships. Given its computational advantages along with the ease to estimate the function’s derivative, ReLU has contributed to the growth and extensive application of DL algorithms (Brownlee, 2018, p. 143). ReLU can be problematic in the presence of large negative bias, which can be mitigated through ReLU extensions (Brownlee, 2018). Since our target variable, COVID-19 incidence rate per 100,000 population (Jan-Mar 2020) (IR), is non-negative, ReLU appears to be a suitable activation function.

We apply regularization techniques to decrease the variance and avoid overfitting. The most common regularization techniques are: L1, L2 and dropout. The main idea is to penalize model complexity through weights (Brownlee, 2018, p. 247). Both L1 and L2 constrain weight values by adding a regularization term (penalty) in the loss function (Tikhonov, 1943), such that the model learns to decrease the weight values until the optimal weights and loss are achieved. In contrast, dropout constrains the number of weights without adding any regularization term in the loss function.

Dropout has been widely adopted, because it not only regularizes a model, but also makes it more robust since it offers the possibility to systematically consider numerous parsimonious neural networks through the randomized procedure applied to drop hidden units (Phaisangittisagul, 2016). We therefore select the dropout approach using default dropout rates (0.5-0.8) (Garbin, Zhu, and Marques, 2020) and explore additional values between 0.1 to 0.9 to ensure that we select the most suitable rate for our case study.

In order to decrease the training time and improve the model performance and stability, we adopt a batch normalization (BN) technique, which normalizes z values in hidden layers. In addition, BN has a slight regularization effect and decreases the impact of weight initialization on model performance and has been widely used (Ioffe and Szegedy, 2015).

## Model optimizatation

As a means to optimize the MLP models, various gradient optimization algorithms can be used. They commonly perform backpropagation to adjust the weights and biases of units and minimize the loss function of neural network models (Rumelhart, Hinton, and Williams, 1986). For this purpose, derivatives of the loss function are calculated and evaluated. Subsequently, gradients of weights and biases are passed to the units from the last to the first layer (backwards) and adjusted with the learning rate such that the model accuracy is improved.

Popular algorithms include the batch gradient descent, gradient descent (GD), and additional versions which use different batch size, i.e. number of training examples that go through a neural network in each iteration and different criteria to set the timing from which loss and derivative calculations are performed. As a means to decrease training time while achieving better optima, more sophisticated algorithms have been proposed, which include: an extension of GD: SGD with momentum, root mean square propagation (RMSProp), and adaptive moment estimation (Adam). These algorithms differ only by calculating and scaling the derivative terms to find the optimum. Rumelhart, Hinton, and Williams (1986) proposed SGD with momentum, which uses exponentially weighted averages to smooth the noise (oscillations) that might prevent or increase the necessary time for the model to reach the optimum.

Adam combines the benefits of both SGD with momentum and RMSProp algorithms and has achieved good results in various studies (Kingma and Ba, 2015). Therefore, we chose Adam as optimization algorithm. In order to increase the chance that an optimum is reached while keeping the computational time reasonable, we explore a range of plausible values for the learning rate by taking suggestions from the literature. We initiate the learning rate with a value 0.01 and decrease it for each iteration.

Batch size describes how many training examples go through a neural network in each iteration. We define an initial batch size of 64 and explore additional batch sizes: 32, 96 and 128. Depending on batch size there is a trade-off between model generalisability and training time. Hence, these batch sizes are optimal for this study (Goodfellow, Bengio, and Courville, 2016). In addition, we need to set the epoch, which is defined as a single pass through the full training set examples (Nielsen, 2015). More epochs are needed to train a model with a low learning rate. We set a relatively high number of epochs (10,000) and added an early stopping algorithm to stop the training process before the maximum number of epochs is reached if the validation loss does not improve further (Prechelt, 1998).

In [14]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, BatchNormalization, Dropout, Activation, InputLayer
from tensorflow.keras.optimizers import Adam
from kerastuner.tuners import RandomSearch
from tensorflow.keras import metrics
from tensorflow.keras.callbacks import EarlyStopping

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from utils import ROOT_DIR, get_data, plot_predicted_vs_true, custom_r2, custom_adj_r2
from shutil import rmtree

%load_ext autoreload
%autoreload 2

%matplotlib inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [15]:
model_configs= {
    "hidden_layers_0": {
        "n_layers": 0,
        "random_search_project_name": "covid-19-nn-0",
        "random_seed": 5959,
        "model_save_path": ROOT_DIR + "/best-models/mlp/hidden-layers-0",
    },
    "hidden_layers_1": {
        "n_layers": 1,
        "random_search_project_name": "covid-19-nn-1",
        "random_seed": 1184,
        "model_save_path": ROOT_DIR + "/best-models/mlp/hidden-layers-1",
    },
    "hidden_layers_2": {
        "n_layers": 2,
        "random_search_project_name": "covid-19-nn-2",
        "random_seed": 4442,        
        "model_save_path": ROOT_DIR + "/best-models/mlp/hidden-layers-2",
    }
}

In [16]:
# This set seeds to make the result reproducible
# reproduce(0)

# read in data using pandas
# X_train, Y_train, X_validation, Y_validation, X_test, Y_test = get_data()

Xs=[X_train, X_validation, X_test]
Ys=[Y_train, Y_validation, Y_test]

# get number of columns in training data
n_cols = X_train.shape[1]

max_trial = 1000

early_stopping_monitor = EarlyStopping(monitor="val_loss", patience=100)

In [17]:
def build_model(hp):
    model = Sequential()
    model.add(InputLayer(input_shape=(n_cols,)))
    
    if current_model_config["n_layers"] >= 1:
        num_units_per_layer = hp.Int('units_per_hidden_layer', min_value=32,
                                     max_value=4096, step=32)
        dropout_rate_per_layer = hp.Float('dropout_per_hidden_layer', min_value=0.0,
                                          max_value=0.9, step=0.1, default=0.5)

        for i in range(hp.Choice('num_layers', values=[current_model_config["n_layers"]])):
            model.add(Dense(units=num_units_per_layer,
                            kernel_initializer='he_uniform',
                            bias_initializer='zeros'))
            model.add(BatchNormalization())
            model.add(Activation("relu"))
            model.add(Dropout(rate=dropout_rate_per_layer))

    model.add(Dense(1, 
                    kernel_initializer='he_uniform',
                    bias_initializer='zeros'))
    model.add(BatchNormalization())
    model.add(Activation("relu"))
    
    model.compile(optimizer=Adam(hp.Choice('learning_rate',
                                           values=[0.01, 0.005, 0.0025, 0.00125, 0.000625, 0.0003125])),
                  loss='mean_squared_error',
                  metrics=[metrics.RootMeanSquaredError(), metrics.MeanAbsoluteError(), metrics.MeanAbsolutePercentageError(), metrics.MeanSquaredLogarithmicError()])
    return model

In [18]:
# It should have a value False, otherwise it will start to really run Training,
# which will probably take several days
TRAINING = False

tuners = {}

for model_type in ['hidden_layers_0', 'hidden_layers_1', 'hidden_layers_2']:
    current_model_config = model_configs[model_type]
    
    tuners[model_type] = RandomSearch(
        build_model,
        seed=current_model_config['random_seed'],
        objective="val_loss",
        max_trials=max_trial,
        executions_per_trial=3,
        directory=ROOT_DIR + "/random-search",
        project_name=current_model_config["random_search_project_name"])

    if TRAINING:
        # Use .values to convert pandas dataframe to numpy array
        # To avoid the Warning -> WARNING:tensorflow:Falling back from v2 loop because of error: Failed to find data adapter that can handle input: <class 'pandas.core.frame.DataFrame'>, <class 'NoneType'>
        tuners[model_type].search(X_train.values, Y_train.values,
                     epochs=10000,
                     batch_size=96,
                     validation_data=(X_validation.values, Y_validation.values),
                     verbose=0,
                     callbacks=[early_stopping_monitor])
        
    print("\n\n######## Get the best trained model of '{}'  ######## \n\n".format(model_type.replace("hidden_layers_", "MLP-")))
    best_model_structure_restored = tuners[model_type].get_best_models()[0]
    print(best_model_structure_restored.summary())
    print(tuners[model_type].results_summary(1))
    

INFO:tensorflow:Reloading Oracle from existing project /all/random-search/covid-19-nn-0/oracle.json
INFO:tensorflow:Reloading Tuner from /all/random-search/covid-19-nn-0/tuner0.json


######## Get the best trained model of 'MLP-0'  ######## 


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 1)                 8         
_________________________________________________________________
batch_normalization (BatchNo (None, 1)                 4         
_________________________________________________________________
activation (Activation)      (None, 1)                 0         
Total params: 12
Trainable params: 10
Non-trainable params: 2
_________________________________________________________________
None


None
INFO:tensorflow:Reloading Oracle from existing project /all/random-search/covid-19-nn-1/oracle.json
INFO:tensorflow:Reloading Tuner from /all/random-search/covid-19-nn-1/tuner0.json


######## Get the best trained model of 'MLP-1'  ######## 


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 64)                512       
_________________________________________________________________
batch_normalization (BatchNo (None, 64)                256       
_________________________________________________________________
activation (Activation)      (None, 64)                0         
_________________________________________________________________
dropout (Dropout)            (None, 64)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 65        
_____

None
INFO:tensorflow:Reloading Oracle from existing project /all/random-search/covid-19-nn-2/oracle.json
INFO:tensorflow:Reloading Tuner from /all/random-search/covid-19-nn-2/tuner0.json


######## Get the best trained model of 'MLP-2'  ######## 


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 128)               1024      
_________________________________________________________________
batch_normalization (BatchNo (None, 128)               512       
_________________________________________________________________
activation (Activation)      (None, 128)               0         
_________________________________________________________________
dropout (Dropout)            (None, 128)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 128)               16512     
_____

None


# Results

## Model selection

Using a total of 812 observations (training set: 486 observations, validation set (163 observations), and test set (163 observations), we compare the predictive performance and select the best model by comparing the root mean squared error (RMSE) metric computed for three parsimonious multilayer perceptron models (initialized with 128 hidden units): (1) without a hidden layer (MLP0); (2) with one hidden layer (MLP1); and (3) with two hidden layers (MLP2). Table 1 provides the RMSE estimated for training, validation and test sets of the three investigated models.

We select MLP1 since it performs best on the training, validation and test sets (bold values in Table 1). Hyperparameters of MLP models are optimized by random search algorithms and training is stopped by early stopping algorithm, when RMSE does not improve on validation set. The optimal hyperparameters of the selected model are 64 hidden units, 0.8 dropout rate, 0.005 learning rate and a batch size of size 96 and 800 epochs. To ensure R & R, note that we include a random seed (equal to 1184) associated with the weight initialization.

Table 1.: The root mean squared error (RMSE) metric is computed for the training, validation, and test sets of three multilayer perceptron (MLP) models with : 0 hidden layer (MLP0), 1 hidden layer (MLP1), and 2 hidden layers (MLP2). The target variable is COVID-19 incidence rate per 100,000 population (Jan-Mar 2020) (IR). The results of the best performing model are in bold characters.

| Model  | hidden layer | training   | validation | test       |
| ------ | ------------ | ---------- | ---------- | ---------- |
| MLP0   | 0            | 0.70       | 0.68       | 0.66       |
| MLP1   | 1            | ***0.63*** | ***0.62*** | ***0.63*** |
| MLP2   | 2            | 0.68       | 0.63       | 0.66       |
| Nb.obs |              | 486        | 163        | 163        |


## Mapping COVID-19 risk at fine spatial scale

We make fine-scale predictions of COVID-19 risk using the results from the best performing model (MLP1). Figure 2 shows the predicted values of LIR within 5×5 km2 grid-cells (548,255) that cover China. Note that the predicted values are on a natural log scale (dark colors: low values; bright colors: high values). White pixels inside China represent missing values. As expected, the highest LIR values are located in Hubei province, which is the presumed epicenter of the COVID-19 outbreak. The lowest LIR values are predicted in Northwest and Southwest China, which are far away (in both access time and distance) from the epicenter.

To obtain a local prediction of the magnitude of the COVID-19 associated with a given population size, we compute the log COVID-19 total confirmed cases (Jan-Mar 2020) (LTC) at fine spatial scales across China. We take the log of the product of IR with population size for each grid-cell. The results are illustrated in Figure 3. Consistently, the highest predicted values of LTC are located in Hubei province, which counts the largest number of cases in China.

## Model validity

There is no available grid-cell data to assess the performance of predictions in locations that did not provide data. For that reason, we cannot systematically compare the observed and predicted values of TC at fine spatial scales across China. As an alternative, we compare the total confirmed number of cases (January to March 2020) provided by the reference COVID-19 dataset, JHU, with fine-scale predictions of TC aggregated at province-level.

Table A3 provides the TC data from JHU with province-level aggregated fine-scale predictions. Additionally, the table shows raw and percentage differences between the predictions and the external dataset. As expected, the largest discrepancy is observed in Hubei province, where the model underestimates TC, with a large deficit of counts of about 63,147. Given the outlier in Hubei province, an underestimation from the model is almost unavoidable. Also, it is likely that the data used in the model is less comprehensive than JHU, which may partially explain the observed discrepancy. The model slightly overestimates TC in Hebei province, with an added number of 365 cases and predicts 234 cases in Inner Mongolia, while JHU did not observe any cases over the investigated time. In this later case, we hypothesize delays in the reporting of cases. A more recent version (March 6, 2021) of JHU reports a total of 367 cases in Inner Mongolia observed since the initial outbreak in December 2019, which provides support to our hypothesis (JHU CSSE, 2020).

To assess the validity of the model with external data, we compute the RMSE, MAE and R2 metrics associated with the difference in TC between the predictions and JHU data at province level (Table 2). We show the computed metrics based on all provinces (first row) and without Hubei province (second row). Computing the metrics without outlier values (in Hubei) allows us to better capture the overall differences between the predictions of the model and JHU data in all provinces except in Hubei province, which accounts for most differences.

Table 2.: Metrics, root mean squared error (RMSE), mean absolute error (MAE) and R2, calculated for differences between aggregated fine-scale predictions of COVID-19 total confirmed cases (Jan-Mar 2020) (TC) and JHU data (Jan-Mar 2020) at province level in China. Calculations are conducted for 33 (all) provinces including Hubei province and for 32 without Hubei province. Fine-scale values of TC (548,255 in total) were predicted by multilayer perceptron with one hidden layer (MLP1).

|                | RMSE     | MAE     | R<sup>2</sup> |
|----------------|----------|---------|----------------|
| All  provinces | 10995.69 | 2104.67 | 0.094          |
| Without Hubei  | 267.82   | 197.09  | 0.564          |


As expected, the RMSE and MAE based on all provinces including Hubei are one to two orders of magnitude higher than those computed in all provinces except Hubei. The large values of the RMSE compared to MAE obtained when all provinces are considered indicate that one or a few provinces exhibit differences that are much larger than the average differences observed in other provinces, since RMSE puts higher penalty (squared errors) on larger differences. In the assessment that excludes Hubei, the total variance in JHU data is improved, with about 56% of the variance that can be explained by the predictive model.

Figure 4 shows the differences between the predictions of COVID-19 cases and JHU data for each province in China. In particular, it captures a negative difference of 63,147 TC in Hubei province (number 14). The second largest differences are observed in Hebei province, where the predictive values show a positive difference of 365 cases compared to JHU data.

In [94]:
## Plot the image for differences as Figure 4 on p. 18

# Conclusion

In [92]:
a_string = '''As expected, the RMSE and MAE based on all provinces including Hubei are one to
two orders of magnitude higher than those computed in all provinces except Hubei. The
large values of the RMSE compared to MAE obtained when all provinces are considered
indicate that one or a few provinces exhibit differences that are much larger than the
average differences observed in other provinces, since RMSE puts higher penalty (squared
errors) on larger differences. In the assessment that excludes Hubei, the total variance
in JHU data is improved, with about 56% of the variance that can be explained by the
predictive model.

Figure 4 shows the differences between the predictions of COVID-19 cases and JHU
data for each province in China. In particular, it captures a negative difference of 63,147
TC in Hubei province (number 14). The second largest differences are observed in Hebei
province, where the predictive values show a positive difference of 365 cases compared
to JHU data.'''
print(a_string.replace("\n\n", "XXXXXXXX").replace("\n", " ").replace("XXXXXXXX", "\n\n"))

As expected, the RMSE and MAE based on all provinces including Hubei are one to two orders of magnitude higher than those computed in all provinces except Hubei. The large values of the RMSE compared to MAE obtained when all provinces are considered indicate that one or a few provinces exhibit differences that are much larger than the average differences observed in other provinces, since RMSE puts higher penalty (squared errors) on larger differences. In the assessment that excludes Hubei, the total variance in JHU data is improved, with about 56% of the variance that can be explained by the predictive model.

Figure 4 shows the differences between the predictions of COVID-19 cases and JHU data for each province in China. In particular, it captures a negative difference of 63,147 TC in Hubei province (number 14). The second largest differences are observed in Hebei province, where the predictive values show a positive difference of 365 cases compared to JHU data.
